# <center>CryptoCurrencies CA#1<center>
<h1><center>Introduction to Cryptography<center></h1>

#### <center>"Mahdi - Noori"<center>

## StoryLine:
We have TinTin and Haddock on two seperate islands and they want to send messages to each other using morse codes!!    

However, there is a problem over there. Roberto Rastapopoulos (Their enemy!) is listening to their conversation to figure out what they are saying to each other!    

Now TinTin wants to use some encryption and decryption to ensure that Roberto couldn't understand what they are saying. Help TinTin to acheive his goal!

## Imports:

In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256
from Crypto.Signature import PKCS1_v1_5

## Contatnts:

In [5]:
IV = get_random_bytes(16)
AES_MODE = AES.MODE_CBC
MESSAGE_PATH = "Confidential-Message.txt"
RSA_KEY_SIZE = 1024

## Part 1: Symmetric Cryptography



**Important Note**: In all steps, use **AES-128** encryption method for encryption/decryption. You can use python cryptography libraries for this purpose.

TinTin decided to design a symmetirc encryption scheme for his connection with haddock. He asked you to design it for him using these steps:

In [6]:
def symmetric_encryptor(input_message:str , key:str):
    # Write a program to encrypt the given string input
    # CODE HERE:
    
    cipher = AES.new(key, AES_MODE, iv=IV)
    cipher_text = cipher.encrypt(pad(input_message.encode('utf-8'), AES.block_size))

    return cipher_text

In [7]:
def symmetric_decryptor(input_ciphertext:str , key:str):
    # Write a program to decrypt the given string input
    # CODE HERE:
    
    cipher = AES.new(key, AES_MODE, iv=IV)
    plain_text = cipher.decrypt(input_ciphertext)
    
    return unpad(plain_text, AES.block_size).decode('utf-8')

In [8]:
def symmetric_verifier(plaintext:str, ciphertext:str, key:str):
    # Verify the correctness of the decryption process by
    # comparing the original message with the decrypted message.
    # output should be boolean(ture or false)
    # CODE HERE:

    decrypted_text = symmetric_decryptor(ciphertext, key)
    return decrypted_text == plaintext

Now test your functions by encrypting & decrypting the ((confidential-message.txt)) file.

First we generate a key for encryption and open the text file

In [9]:
symmetric_encryption_key = get_random_bytes(16)
with open(MESSAGE_PATH, "r") as file:
    message = file.read()

# print(message)

In [10]:
# Test encryption code:
# CODE HERE:

encrypted_message = symmetric_encryptor(message, symmetric_encryption_key)

In [11]:
# Test decryption code:
# CODE HERE:

decrypted_message = symmetric_decryptor(encrypted_message, symmetric_encryption_key)

In [12]:
# Test verification code:
# CODE HERE:

print(symmetric_verifier(message, encrypted_message, symmetric_encryption_key))

True


## Part 2: Asymmetric Cryptography

After TinTin found how to use symmetric encryption using a key, he faced another problem! How to share a key with haddock without letting Roberto finds it?        

Therefore, he decides to use an asymmetric crypto system.    
help him to acheive his goal by writing a program which:

In [14]:
def RSA_key_generator():
    # write a code to generate a pair of private/public key using RSA
    # CODE HERE:
    
    key = RSA.generate(RSA_KEY_SIZE)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key
    

In [15]:
def asymmteric_encryptor(public_key:str, input_message:str):
    # write a code to encrypt an input message using your public key
    # CODE HERE:

    key = RSA.importKey(public_key)
    cipher = PKCS1_OAEP.new(key)
    cipher_text = cipher.encrypt(input_message.encode('utf-8'))
    return cipher_text

In [16]:
def asymmteric_decryptor(private_key:str, ciphertext:str):
    # write a code to decrypt a ciphertext using your private key
    # CODE HERE:

    key = RSA.importKey(private_key)
    cipher = PKCS1_OAEP.new(key)
    plain_text = cipher.decrypt(ciphertext)
    return plain_text.decode('utf-8')

In [17]:
def asymmetric_verifier(plaintext:str, ciphertext:str, private_key:str):
    # Verify the correctness of the decryption process by
    # comparing the original message with the decrypted message.
    # output should be boolean(ture or false)
    # CODE HERE:
    
    decrypted_text = asymmteric_decryptor(private_key, ciphertext)
    return decrypted_text == plaintext

Now test your functions by encrypting & decrypting the symmetric key of previous part using new asymmetric functions.

First we generate the keys

In [18]:
private_key, public_key = RSA_key_generator()

In [27]:
# Test encryption code:
# CODE HERE:

encrypted_message = asymmteric_encryptor(public_key, str(symmetric_encryption_key))

In [29]:
# Test decryption code:
# CODE HERE:

decrypted_message = asymmteric_decryptor(private_key, encrypted_message)

In [30]:
# Test verification code:
# CODE HERE:

print(asymmetric_verifier(str(symmetric_encryption_key), encrypted_message, private_key))

True


## Part 3: Digital signature

Write a function to sign a given message in the text file (Confidentail message) using private key which was made in the part 2.   
You have to sign the hash of the message, not the whole message (why?). (Use SHA-256 for hashing)

1. `Data Size Limitation:`<br>

    RSA operations work with integers modulo a given integer (the modulus, part of the public key). The size of the modulus determines the maximum data size that RSA can handle. By hashing the message, we reduce its size to fit within the RSA limits.

2. `Security Considerations:`<br>

    - An adversary could manipulate the message and adopt the signature by swapping or dropping sub-messages. Hashing prevents this vulnerability.

    - An adversary can randomly choose a signature and compute a corresponding message m. Hashing and padding prevent such forgeries.

So we can say that hashing before signing ensures data fits within RSA limits and provides security guarantees against manipulation and forgery.

In [49]:
def sign_message(private_key:str, input_message: str):
    # Write a code to sign a message using your private key.
    # CODE HERE:
    
    digest = SHA256.new()
    digest.update(input_message.encode('utf-8'))

    key = RSA.importKey(private_key)

    signer = PKCS1_v1_5.new(key)
    message_signature = signer.sign(digest)
    
    return message_signature

In [53]:
def message_verification(input_message:str, signature:str, public_key:str):
    # Write a code to verify the signature of the input message
    # output should be boolean(ture or false)
    # CODE HERE:


    digest = SHA256.new()
    digest.update(input_message.encode('utf-8'))

    key = RSA.importKey(public_key)
    verifier = PKCS1_v1_5.new(key)
    return verifier.verify(digest, signature)


In [54]:
# Test sign code:
# CODE HERE:

signature = sign_message(private_key, message)

In [55]:
# Test verification code:
# CODE HERE:

print(message_verification(message, signature, public_key))

True


## Q3.

- `MAC:`<br>

1. A MAC is a small code that allows the receiver of a message to verify the authenticity of the sender.

2. It is calculated using a message and a secret key as inputs.

3. The sender generates the MAC, and the receiver can verify it using the same secret key.

4. Commonly, a hash function (such as SHA-256) is used to create the MAC.

- `HMAC:`<br>

1. An HMAC is a specific kind of MAC.

2. The main difference lies in the construction. HMAC uses two rounds of hashing (instead of one or none). Each round of hashing involves a section of the secret key.
Example of HMAC calculation:

HMAC enhances security by using double hashing and preventing length extension attacks.